In [3]:
from scipy import stats

# 要进行KS检验的数据列表
data = [1,2,3,4,3,2,1]

# 执行KS检验
statistic, p_value = stats.kstest(data, 'norm')

# 打印结果
print("KS Statistic:", statistic)
print("P-value:", p_value)

# 判断是否符合正态分布
alpha = 0.05  # 显著性水平
if p_value > alpha:
    print("样本数据符合正态分布")
else:
    print("样本数据不符合正态分布")


KS Statistic: 0.8413447460685429
P-value: 5.060890253444183e-06
样本数据不符合正态分布


In [5]:
from steam_reviews import ReviewLoader
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from scikit_posthocs import posthoc_dunn


appid_list = [
211420,
570940,
236430,
335300,
374320,
814380,
1245620,
485510,
1325200,
1448440,
1172380,
1774580,
265300,
678960,
606280,
378540,
644830
]

language_list = ['english', 'schinese', 'russian']

for appid in appid_list:
    group = []
    for language in language_list:
        file_path_p = 'reviews_' + str(appid) + '_' + language + '_p.json'
        file_path_n = 'reviews_' + str(appid) + '_' + language + '_n.json'
        reviews_p = ReviewLoader().load_from_local(file_path_p)
        reviews_n = ReviewLoader().load_from_local(file_path_n)
        
        print(str(appid))
#         print(str(appid) + ', number of positive reviews and negative reviews in ' + language)
#         print(len(reviews_p))
#         print(len(reviews_n))
        
        # review_list_p = reviews_p.review_list()
        playtime_list_p = []
        playtime_list_n = []
        for review in reviews_p.review_dict():
            playtime_list_p.append(review['author']['playtime_forever'] / 60)
            
#             if 'playtime_at_review' in review['author']:
#                 playtime_list_p.append(review['author']['playtime_at_review'] / 60)
            
        
        for review in reviews_n.review_dict():
            playtime_list_n.append(review['author']['playtime_forever'] / 60)
            
#             if 'playtime_at_review' in review['author']:
#                 playtime_list_n.append(review['author']['playtime_at_review'] / 60)
        
        add_list = playtime_list_p + playtime_list_n
        print('Average playtime: ' + str(np.array(add_list).mean()), str(np.array(add_list).std()))
        print('Median playtime: ' + str(np.median(np.array(add_list))))
        # print('Average positive playtime: ' + str(np.array(playtime_list_p).mean() / 60))
        # print('Average negative playtime: ' + str(np.array(playtime_list_n).mean() / 60))
        group.append(add_list)
        
#         # 执行KS检验
#         statistic, p_value = stats.shapiro(add_list)

#         # 打印结果
#         print("KS Statistic:", statistic)
#         print("P-value:", p_value)

#         # 判断是否符合正态分布
#         alpha = 0.05  # 显著性水平
#         if p_value > alpha:
#             print("样本数据符合正态分布")
#         else:
#             print("样本数据不符合正态分布")
        
    print(len(group))
    
    # 执行单因素方差分析
    h_statistic, p_value = stats.kruskal(group[0], group[1], group[2])

    # 打印结果
    print("H-statistic:", h_statistic)
    print("P-value:", p_value)

    # 判断是否具有统计显著性
    alpha = 0.05  # 显著性水平
    # 如果Kruskal-Wallis检验显著，则执行Dunn's多重比较检验
    if p_value < 0.05:  # 根据需要调整显著性水平
        # 合并数据以进行多重比较
        data = np.concatenate([group[0], group[1], group[2]])
        # 创建相应的组标签
        labels = ['A'] * len(group[0]) + ['B'] * len(group[1]) + ['C'] * len(group[2])

        # 使用MultiComparison执行Dunn's检验
        mc = MultiComparison(data, labels)
        result_dunn = mc.allpairtest(stats.kruskal, method='bonf')
        print(result_dunn[0])  # 打印事后检验结果
        # 进行Dunn's多重比较
#         dunn_results = posthoc_dunn([group[0], group[1], group[2]], p_adjust='bonferroni')

#         # 打印Dunn's检验结果
#         print(dunn_results)

211420
Average playtime: 146.5189569319472 345.97195399458326
Median playtime: 94.58333333333333
211420
Average playtime: 118.56564892623717 460.8074573484599
Median playtime: 73.82499999999999
211420
Average playtime: 129.50649407987157 159.37315302243235
Median playtime: 90.81666666666666
3
H-statistic: 206.08463299128536
P-value: 1.7753768124283902e-45
Test Multiple Comparison kruskal 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat  pval pval_corr reject
-------------------------------------------
     A      B 191.073  0.0       0.0   True
     A      C 27.9309  0.0       0.0   True
     B      C  79.288  0.0       0.0   True
-------------------------------------------
570940
Average playtime: 88.89911846480047 120.85273865796506
Median playtime: 60.81666666666667
570940
Average playtime: 79.64358800418327 243.53826073289986
Median playtime: 53.766666666666666
570940
Average playtime: 84.25127345011802 108.16316568415245
Median playtime: 59.9
3
H-stat

1774580
Average playtime: 41.980738812819844 39.352498684662734
Median playtime: 35.43333333333333
1774580
Average playtime: 39.23517880224042 70.51492826526794
Median playtime: 33.2
1774580
Average playtime: 41.381706435285615 41.00698094845823
Median playtime: 34.86666666666667
3
H-statistic: 26.835390481505016
P-value: 1.488569091429574e-06
Test Multiple Comparison kruskal 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
     A      B 26.4348    0.0       0.0   True
     A      C  0.6744 0.4115       1.0  False
     B      C  2.0595 0.1513    0.4538  False
---------------------------------------------
265300
Average playtime: 21.036364285714285 35.39458134448581
Median playtime: 12.933333333333334
265300
Average playtime: 17.37643064985451 97.94328557035386
Median playtime: 6.266666666666667
265300
Average playtime: 24.564610866372977 37.041885373032834
Median playtime: 15.85
3
H-sta